# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import geoapify_key
print(geoapify_key)

860ed6f9e0614c61aa8a2fa77ffa8e0b


In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,walvis bay,-22.9575,14.5053,60.10,82,69,7.00,NaN,1701042164
1,1,olonkinbyen,70.9221,-8.7187,34.16,85,100,18.23,SJ,1701042164
2,2,bethel,41.3712,-73.4140,48.65,82,100,10.36,US,1701041957
3,3,saint-francois,46.4154,3.9054,31.64,94,100,2.30,FR,1701042040
4,4,grytviken,-54.2811,-36.5092,38.77,92,99,10.74,GS,1701042021


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

city_data_df = city_data_df.dropna()

city_data_df
# Store 'Lat' and 'Lng' into  locations 
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

# Create a Humidity heatmap layer
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100, 
                                 point_radius = 2)

fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) 
                                    & (city_data_df["Max Temp"] > 70) 
                                    & (city_data_df["Wind Speed"] < 10) 
                                    & (city_data_df["Cloudiness"] == 0),:].dropna()
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,punta del este,-34.9667,-54.9500,75.31,77,0,9.22,UY,1701042169
169,169,saint-pierre,-21.3393,55.4781,71.56,68,0,6.91,RE,1701042194
234,234,giyani,-23.3025,30.7187,73.96,49,0,4.41,ZA,1701042204
309,309,tsiombe,-25.3000,45.4833,70.97,96,0,4.74,MG,1701042215
381,381,maragogi,-9.0122,-35.2225,76.57,71,0,5.35,BR,1701041982


### Step 3: Create a new DataFrame called `hotel_df`.

In [45]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
19,punta del este,UY,-34.9667,-54.9500,
169,saint-pierre,RE,-21.3393,55.4781,
234,giyani,ZA,-23.3025,30.7187,
309,tsiombe,MG,-25.3000,45.4833,
381,maragogi,BR,-9.0122,-35.2225,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
params = {
    "radius": 10000,
    "type": "lodging",
    "key": geoapify_key}



# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # Make and API request using the params dictionaty
    #name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
punta del este - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
giyani - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
maragogi - nearest hotel: No hotel found
veraval - nearest hotel: No hotel found
sao jose da coroa grande - nearest hotel: No hotel found
santo antonio - nearest hotel: No hotel found
ratnagiri - nearest hotel: No hotel found
koundara - nearest hotel: No hotel found


,City,Country,Lat,Lng,Hotel Name
19,punta del este,UY,-34.9667,-54.9500,No hotel found
169,saint-pierre,RE,-21.3393,55.4781,No hotel found
234,giyani,ZA,-23.3025,30.7187,No hotel found
309,tsiombe,MG,-25.3000,45.4833,No hotel found
381,maragogi,BR,-9.0122,-35.2225,No hotel found
390,veraval,IN,20.9000,70.3667,No hotel found
419,sao jose da coroa grande,BR,-8.8978,-35.1478,No hotel found
444,santo antonio,BR,-6.3106,-35.4789,No hotel found
455,ratnagiri,IN,16.9833,73.3000,No hotel found
540,koundara,GN,12.4833,-13.3000,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)